In [1]:
#import fastai
from fastai import *          # Quick access to most common functionality
from fastai.vision import *   # Quick access to computer vision functionality
from fastai.callbacks import *

In [2]:
import pytorch_ssim as ssim
from superres import *
from torchvision.models import vgg16_bn
import czifile

In [39]:
path = Path('/DATA/WAMRI/salk/uri/BPHO/')
path_raw = path/'raw'
path_processed = path/'processed'
path_input = path/'input'
path_target = path/'target'

path_input.mkdir(exist_ok=True)
path_target.mkdir(exist_ok=True)

def to_airyscan(fn):
    new_fn = fn.stem + '_Airyscan Processing' + fn.suffix
    new_fn = Path(new_fn)
    new_parent = (path_processed/fn.relative_to(path_raw)).parent
    return new_parent/new_fn

In [40]:
raw_files = list(path_raw.glob('*'))
path_raw, len(raw_files)
fn = raw_files[0]

In [43]:
def get_czi_shape_info(czi):
    shape = czi.shape
    axes = czi.axes
    axes_dict = {axis:idx for idx,axis in enumerate(czi.axes)}
    shape_dict = {axis:shape[axes_dict[axis]] for axis in czi.axes}
    return axes_dict, shape_dict


def build_index(axes, ix_select):
    idx = [ix_select.get(ax, 0) for ax in axes]
    return tuple(idx)

def process_czi(raw_fn):
    proc_fn = to_airyscan(raw_fn)
    if not proc_fn.exists():
        print(f'no match: {proc_fn}')
        return
    
    with czifile.CziFile(fn) as raw_czf:
        raw_axes, raw_shape = get_czi_shape_info(raw_czf)
        channels = raw_shape['C'] // 2
        sensors = raw_shape['H']
        depths = raw_shape['Z']
        x,y = raw_shape['X'], raw_shape['Y']
        data = raw_czf.asarray()
        for channel in range(channels):
            for depth in range(depths):
                for sensor in range(sensors):
                    idx = build_index(raw_axes, {'H':sensor, 'C': channel, 'Z':depth,
                                                 'X':slice(0,x),'Y':slice(0,y)})
                    img = data[idx]
                    save_raw_fn = path_input/f'{raw_fn.stem}_{channel:02d}_{depth:03d}_{sensor:02d}.npy'
                    np.save(save_raw_fn, img)
                    
    with czifile.CziFile(proc_fn) as proc_czf:
        proc_axes, proc_shape = get_czi_shape_info(proc_czf)
        channels = proc_shape['C']
        depths = proc_shape['Z']
        x,y = proc_shape['X'], proc_shape['Y']
        data = proc_czf.asarray()
        for channel in range(channels):
            for depth in range(depths):
                for sensor in range(sensors):
                    idx = build_index(proc_axes, {'C': channel, 'Z':depth, 'X':slice(0,x),'Y':slice(0,y)})
                    img = data[idx]
                    save_proc_fn = path_target/f'{raw_fn.stem}_{channel:02d}_{depth:03d}.npy'
                    np.save(save_proc_fn, img)
        

In [45]:
#for fn in progress_bar(raw_files):
#    process_czi(fn)